# Mujoco Payload Lifting Controller Example 
This example load a basic robot model(i.e. composed by basic shapes), modify the links of the robot by elongating the legs, define instances of payload liftin planner and istnataneous controller and simulates the robot behavior with mujoco .

In [ ]:
# Comodo import
from comodo.mujocoSimulator.mujocoSimulator import MujocoSimulator
from comodo.robotModel.robotModel import RobotModel
from comodo.robotModel.createUrdf import createUrdf
from comodo.ergonomyPlanner.planErgonomyTrajectory import PlanErgonomyTrajectory
from comodo.payloadLiftingController.payloadLiftingController import PayloadLiftingController
from comodo.payloadLiftingController.payloadLiftingParameterTuning import PayloadLiftingControllerParameters

In [ ]:
# General  import
import xml.etree.ElementTree as ET
import numpy as np
import tempfile
import urllib.request

In [ ]:
# Getting stickbot urdf file and convert it to string
urdf_robot_file = tempfile.NamedTemporaryFile(mode="w+")
url = "https://raw.githubusercontent.com/icub-tech-iit/ergocub-gazebo-simulations/master/models/stickBot/model.urdf"
urllib.request.urlretrieve(url, urdf_robot_file.name)
# Load the URDF file
tree = ET.parse(urdf_robot_file.name)
root = tree.getroot()

# Convert the XML tree to a string
robot_urdf_string_original = ET.tostring(root)

create_urdf_instance = createUrdf(
    original_urdf_path=urdf_robot_file.name, save_gazebo_plugin=False
)

In [ ]:
# Define parametric links and controlled joints
legs_link_names = ["hip_3", "lower_leg"]
joint_name_list = [
    "r_shoulder_pitch",
    "r_shoulder_roll",
    "r_shoulder_yaw",
    "r_elbow",
    "l_shoulder_pitch",
    "l_shoulder_roll",
    "l_shoulder_yaw",
    "l_elbow",
    "r_hip_pitch",
    "r_hip_roll",
    "r_hip_yaw",
    "r_knee",
    "r_ankle_pitch",
    "r_ankle_roll",
    "l_hip_pitch",
    "l_hip_roll",
    "l_hip_yaw",
    "l_knee",
    "l_ankle_pitch",
    "l_ankle_roll",
]

In [ ]:
# Define the robot modifications
modifications = {}
for item in legs_link_names:
    left_leg_item = "l_" + item
    right_leg_item = "r_" + item
    modifications.update({left_leg_item: 1.2})
    modifications.update({right_leg_item: 1.2})
# Motors Parameters
Im_arms = 1e-3 * np.ones(4)  # from 0-4
Im_legs = 1e-3 * np.ones(6)  # from 5-10
kv_arms = 0.001 * np.ones(4)  # from 11-14
kv_legs = 0.001 * np.ones(6)  # from 20

Im = np.concatenate((Im_arms, Im_arms, Im_legs, Im_legs))
kv = np.concatenate((kv_arms, kv_arms, kv_legs, kv_legs))

In [ ]:
# Modify the robot model and initialize
create_urdf_instance.modify_lengths(modifications)
urdf_robot_string = create_urdf_instance.write_urdf_to_file()
create_urdf_instance.reset_modifications()
robot_model_init = RobotModel(urdf_robot_string, "stickBot", joint_name_list)

In [ ]:
## Planning the ergonomic trajectory for payload lifting 
plan_trajectory = PlanErgonomyTrajectory(robot_model=robot_model_init)

if(not(plan_trajectory.plan_trajectory())):
   raise ValueError("unable to plan the trajectory ")
 

In [ ]:
# Define simulator and set initial position
mujoco_instance = MujocoSimulator()
mujoco_instance.load_model(
    robot_model_init, s=plan_trajectory.s_opti[0], xyz_rpy=plan_trajectory.xyz_rpy[0], kv_motors=kv, Im=Im
)
s, ds, tau = mujoco_instance.get_state()
t = mujoco_instance.get_simulation_time()
H_b = mujoco_instance.get_base()
w_b = mujoco_instance.get_base_velocity()
mujoco_instance.set_visualize_robot_flag(True)

In [ ]:
## Defining controller 
lifting_controller_instance = PayloadLiftingController(frequency=0.01, robot_model=robot_model_init)
lifting_controller_instance.set_state(s,ds,t)
param_lifting_controller = PayloadLiftingControllerParameters()
lifting_controller_instance.set_control_gains(
postural_Kp=param_lifting_controller.joints_Kp_parameters,
CoM_Kp=param_lifting_controller.CoM_Kp,
CoM_Ki=param_lifting_controller.CoM_Ki,
)
lifting_controller_instance.set_time_interval_state_machine(10, 40, 40)
lifting_controller_instance.initialize_state_machine(
    joint_pos_1=plan_trajectory.s_opti[1], joint_pos_2=plan_trajectory.s_opti[2]
)


In [ ]:
n_step = int(lifting_controller_instance.frequency/mujoco_instance.get_simulation_frequency())
TIME_TH = 75

while(t<TIME_TH):     
    # Updating the states 
    s,ds,tau = mujoco_instance.get_state()
    t = mujoco_instance.get_simulation_time()
    lifting_controller_instance.set_state(s,ds,t)

    # Running the controller 
    controller_succed= lifting_controller_instance.run()
    if(not(controller_succed)): 
        break
    tau = lifting_controller_instance.get_torque()
    mujoco_instance.set_input(tau)
    mujoco_instance.step(int(n_step))

In [ ]:
# Closing visualization
mujoco_instance.close_visualization()